# Principles for Testable Code

- A big part of code design is writing code that is **testable**

- We'll summarise a few factors for testable vs non testable code, just to build up some code quality instincts

## 1. Hidden Dependencies


- Code should be written in a way that all dependencies are explicitly passed in, and accessible at runtime

- This gives you the flexibility of testing different inputs!

In [ ]:
### Bad code
class ABC:
    def submit(self, something):
        db = Database() ### HARDCODED!!! You are not able to test this without accessing the internals of the `submit` method
        db.save(something)

### Better code
class ABC:
    def __init__(self, db):
        self.db = db

    def submit(self, something):
        self.db.save(something)

## 2. Side effects mixed with logic

- When you write code, don't mix in effects with logical processing, it can make things super hard to test

In [ ]:
### Bad code
def process_and_save(data):
    result = data * 2 + 123
    with open('something.txt', 'w') as f:
        f.write(str(result))

### Better code
def process(data):
    return data * 2 + 123

def save(content, path):
    with open(path, 'w') as f:
        f.write(str(content))

## 3. Relying on globals/implicit values

- As far as possible, you NEVER want your classes/functions to reference global values/states, because this makes it riduclously coupled, and if you ever need to change global states in future, you can get failures in unrelated parts of your codebase

In [ ]:
GLOBAL_VAL = 123

### Bad code
def process(data):
    return data * GLOBAL_VAL

### Better code
def process(data, val):
    return data * val

## 4. Non-Deterministic Code

- As far as possible, functions should always return deterministic values

- In the example below, we don't want to run `time.time()` each time; instead, use a `self.clock` input where we can supply a "mock clock" that gives the same result during testing

In [2]:
import time
def time():
    return f"T-{int(time.time())}"

In [3]:
class Timer:
    def __init__(self, clock):
        self.clock = clock

    def generate(self):
        return f"T-{int(self.clock.now())}"

## 5. Tight Coupling

- Don't mix unnecessary things together. This is a general principle, we should think about separating operations wherever possible

In [ ]:
### Bad code, mixing logic with DB write
def process_order(order_id):
    order = db.load(order_id)
    if order.total > 100:
        db.save_discount(order, 0.2)

### Good code
def apply_discount(order):
    if order.total > 100:
        return 0.2
    return 0.0

order = db.load(order_id)
discount = apply_discount(order)
db.save_discount(order, discount)

## 6. Inversion of Control

- You have an object that does something. 
- But the "doing something" code is not contained in the object itself. Instead, it calls something else
- The act of "calling something else" is not testable because:
    1. It is internal to the object; I cannot tell the object what operations to perform when testing
    2. I have zero clue what the call is doing in the context of the object's call

In [5]:
### Bad code
class Button:
    def on_click(self):
        log_event("clicked")
        do_work() ##do_work is defined somewhere else. I cannot change out this "on_click" operation in my test

### Better code
class Button:
    def __init__(self, on_click_handler):
        self.on_click_handler = on_click_handler

    def on_click(self):
        self.on_click_handler() ##on_click_handler can be mocked, and/or replaced with a spy function, during testing